In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [8]:
core_profiles = FUSE.core_profiles();
resize!(core_profiles.profiles_1d,1)
profiles_1d = core_profiles.profiles_1d[1]

profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=21)
profiles_1d.electrons.density = (1.0 .- profiles_1d.grid.rho_tor_norm.^2);
profiles_1d.electrons.temperature = (x;_...) -> (1.0 .- x.^2);

println(profiles_1d.electrons.pressure)
profiles_1d.grid.rho_tor_norm = range(0.0, 1.0, length=5)
println(profiles_1d.electrons.temperature)


ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber


┌ Warning: skipping callee #wall__temperature_reference#258 (called by wall__temperature_reference) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__ids_properties__version_put#259 (called by wall__ids_properties__version_put) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__ids_properties#260 (called by wall__ids_properties) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__global_quantities__neutral___element#261 (called by wall__global_quantities__neutral___element) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292


ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber


┌ Warning: skipping callee #wall__global_quantities__neutral#262 (called by wall__global_quantities__neutral) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__global_quantities__electrons#263 (called by wall__global_quantities__electrons) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__global_quantities#264 (called by wall__global_quantities) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #wall__first_wall_power_flux_peak#265 (called by wall__first_wall_power_flux_peak) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warn

ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber


...

ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber


┌ Warning: skipping callee #core_profiles__profiles_1d___ion___density_fit__time_measurement_slice_method#1050 (called by core_profiles__profiles_1d___ion___density_fit__time_measurement_slice_method) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__profiles_1d___ion___density_fit#1051 (called by core_profiles__profiles_1d___ion___density_fit) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__profiles_1d___ion#1052 (called by core_profiles__profiles_1d___ion) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__profiles_1d___grid#1053 (called by core_profiles__profiles_1d___grid) due to Ass

ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber
ref = :(_1)
typeof(ref) = Core.SlotNumber


┌ Warning: skipping callee #core_profiles__profiles_1d#1061 (called by core_profiles__profiles_1d) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__ids_properties__version_put#1062 (called by core_profiles__ids_properties__version_put) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__ids_properties#1063 (called by core_profiles__ids_properties) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/signatures.jl:292
┌ Warning: skipping callee #core_profiles__global_quantities#1064 (called by core_profiles__global_quantities) due to AssertionError("ref isa GlobalRef")
└ @ LoweredCodeUtils /Users/meneghini/.julia/packages/LoweredCodeUtils/jZY56/src/sign

LoadError: TypeError: in setfield!, expected Union{Missing, FUSE.AbstractFDArray{Float64, 1}}, got a value of type FUSE.var"#7#9"

In [2]:
abstract type AbstractFDNumber <: Number end

struct FDNumber <: AbstractFDNumber
    func_or_value::Any
    _parent::WeakRef
    _field::Symbol
    _raw::Bool
end

function FDNumber(func_or_value)
    return FDNumber(func_or_value, WeakRef(nothing), :x, true)
end
    
function coordinates(fdv::AbstractFDNumber)
    return []
end

Base.convert(::Type{Number}, x::FDNumber) = x.func_or_value
Base.promote_rule(::Type{FDNumber}, ::Type{Number} ) = Number


promote(FDNumber(5),5)



(5, 5)

In [29]:
a=Array{Float64,0}[]
a[]=5

LoadError: BoundsError: attempt to access 0-element Vector{Array{Float64, 0}} at index []